In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import robot as Robot
import  time

robot_Instance= Robot.Robot_Class(6000,1000,90)

SerialException: [Errno 2] could not open port /dev/ttyACM1: [Errno 2] No such file or directory: '/dev/ttyACM1'

In [4]:

#Changer les action_In et action_Out en queue de dictionaire

In [8]:
Start=time.time()
robot_Instance.Set_Robot_Position(0,0,0)
robot_Instance.Go_To(2000,0,0)


Going to 2000   0 with R 0 level 0
OK: X=70.00 Y=-11.04 A=-0.59 

OK: X=0.00 Y=0.00 A=0.00 

OK: X=2OK: X=2000.00 Y=0.00 A=0.00

OK: D_Profil S=750 A=50 B=100 

OK: M3 H3 S15

{'c': 0, 'e': 0, 'g': 1, 'l': 0, 'm': 0, 'f': 0, 'd': 0, 'k': 0, 'b': 0, 'h': 0, 'j': 0, 'i': 0, 'a': 0}
M0 X0.68 Y-0.01 A0.04 T1 S64

Timeout
End of 0


In [5]:
while((robot_Instance.Wait_For_A_Message(0.2))!=None):
    pass


In [28]:
robot_Instance.Wait_For_A_Message(0.2)